In [1]:
import numpy as np
import torch
from torch import nn, optim
from torchtext import data, datasets
from torchtext.vocab import GloVe
from torchtext.vocab import Vectors
from torch.nn import init
from tqdm import tqdm
import pandas as pd
from torchtext.data import get_tokenizer
from sklearn import preprocessing
import math
from sklearn.metrics import f1_score,precision_score,recall_score

In [15]:
w = torch.tensor([[1.0,0.0,1.0,0.0]])
x = torch.tensor([[1.0,1.0,0.0,0.0],[1.0,0.0,0.0,1.0],[0.0,1.0,1.0,0.0],[0.0,1.0,0.0,1.0]])
y = torch.tensor([])

In [22]:
0.5/2*-70+0.3/2*50+0.2/2*200

10.0

In [13]:
s=torch.nn.Softmax(dim=-1)

In [21]:
(w@x).shape

torch.Size([1, 4])

In [19]:
s(w@x)

tensor([[0.1966, 0.5344, 0.1966, 0.0723]])

In [200]:
window_size=31

In [201]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#LABEL = data.Field(sequential=False,use_vocab=False,preprocessing=lambda x:[int(i) for i in x.split(',')],is_target=True)
LABEL = data.Field(sequential=False,use_vocab=False,is_target=True)
TEXT = data.Field(fix_length=window_size,lower=True,preprocessing=lambda x:x[0].split(','))

F:\Anaconda3\envs\pytorch\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [59]:
train = pd.read_csv('preprocess/train.csv')
test = pd.read_csv('preprocess/test.csv')
dev = pd.read_csv('preprocess/dev.csv')

le = preprocessing.LabelEncoder()
event_subtype = train['event_subtype'].unique()
event_subtype = np.insert(event_subtype,0,'None')
le = le.fit(event_subtype)

none_tag = str(le.transform(np.array(['None'])).item())
for ind, row in train.iterrows():
    trigger_first_tokens = row['trigger'].split()[0]
    trigger_text_first_end = row['trigger_text_start'] + len(trigger_first_tokens)
    
    index = -1
    tokens = []
    token=''
    pre_char = ''
    for count,char in enumerate(row['text']):
        if pre_char != ' ':
            if (char == ' ' or (count+1) == len(row['text'])):
                if (count+1) == len(row['text']):
                    token += char
                tokens += [token]
                if (trigger_first_tokens in token or token in trigger_first_tokens) and abs(count-trigger_text_first_end) < 10:
                    index = len(tokens) - 1
                    break
                token = ''
        if char != ' ':
            token += char
        pre_char = char
    if index == -1:
        print('error')
        break

    tokens = row['text'].split()

    left_index = max(0,index - int((window_size-1)/2))
    right_index = index+int((window_size-1)/2) +1 #超过也没关系
    
    
    text_windows = tokens[left_index:right_index]
    if len(text_windows)!=window_size:#需要padding
        left_pad_num = (window_size -1)/2- index - left_index
        text_windows = [TEXT.pad_token] * int(left_pad_num) + text_windows
        text_windows = text_windows + [TEXT.pad_token] * (window_size - len(text_windows))
    
    if (text_windows[15] in trigger_first_tokens) or (trigger_first_tokens in text_windows[15]):
        pass
    else:
        print('error2')
        break
    
    train.loc[ind,'text'] = ','.join(text_windows)
    #train.loc[ind,'text'] = ','.join([str(TEXT.vocab.stoi[i]) for i in text_windows])
    train.loc[ind,'label'] = str(le.transform([row['event_subtype']]).item())
    #train.loc[ind,'label'] = ','.join([str(le.transform([row['event_subtype']]).item())] * len(row['trigger'].split()))
    
for ind, row in dev.iterrows():
    trigger_first_tokens = row['trigger'].split()[0]
    trigger_text_first_end = row['trigger_text_start'] + len(trigger_first_tokens)
    index = -1
    tokens = []
    token=''
    pre_char = ''
    for count,char in enumerate(row['text']):
        if pre_char != ' ':
            if (char == ' ' or (count+1) == len(row['text'])):
                if (count+1) == len(row['text']):
                    token += char
                tokens += [token]
                if (trigger_first_tokens in token or token in trigger_first_tokens) and abs(count-trigger_text_first_end) < 10:
                    index = len(tokens) - 1
                    break
                token = ''
        if char != ' ':
            token += char
        pre_char = char
    if index == -1:
        print('error')
        break

    tokens = row['text'].split()

    left_index = max(0,index - int((window_size-1)/2))
    right_index = index+int((window_size-1)/2) +1 #超过也没关系
    
    
    text_windows = tokens[left_index:right_index]
    if len(text_windows)!=window_size:#需要padding
        left_pad_num = (window_size -1)/2- index - left_index
        text_windows = [TEXT.pad_token] * int(left_pad_num) + text_windows
        text_windows = text_windows + [TEXT.pad_token] * (window_size - len(text_windows))
    
    if (text_windows[15] in trigger_first_tokens) or (trigger_first_tokens in text_windows[15]):
        pass
    else:
        print('error2')
        break
    
    dev.loc[ind,'text'] = ','.join(text_windows)
    dev.loc[ind,'label'] = str(le.transform([row['event_subtype']]).item())
    #train.loc[ind,'text'] = ','.join([str(TEXT.vocab.stoi[i]) for i in text_windows])
    #dev.loc[ind,'label'] = ','.join([str(le.transform([row['event_subtype']]).item())] * len(row['trigger'].split()))
    
for ind, row in test.iterrows():
    trigger_first_tokens = row['trigger'].split()[0]
    trigger_text_first_end = row['trigger_text_start'] + len(trigger_first_tokens)
    index = -1
    tokens = []
    token=''
    pre_char = ''
    for count,char in enumerate(row['text']):
        if pre_char != ' ':
            if (char == ' ' or (count+1) == len(row['text'])):
                if (count+1) == len(row['text']):
                    token += char
                tokens += [token]
                if (trigger_first_tokens in token or token in trigger_first_tokens) and abs(count-trigger_text_first_end) < 10:
                    index = len(tokens) - 1
                    break
                token = ''
        if char != ' ':
            token += char
        pre_char = char
    if index == -1:
        print('error')
        break

    tokens = row['text'].split()

    left_index = max(0,index - int((window_size-1)/2))
    right_index = index+int((window_size-1)/2) +1 #超过也没关系
    
    
    text_windows = tokens[left_index:right_index]
    if len(text_windows)!=window_size:#需要padding
        left_pad_num = (window_size -1)/2- index - left_index
        text_windows = [TEXT.pad_token] * int(left_pad_num) + text_windows
        text_windows = text_windows + [TEXT.pad_token] * (window_size - len(text_windows))
    
    if (text_windows[15] in trigger_first_tokens) or (trigger_first_tokens in text_windows[15]):
        pass
    else:
        print('error2')
        break
    
    test.loc[ind,'text'] = ','.join(text_windows)
    test.loc[ind,'label'] = str(le.transform([row['event_subtype']]).item())
    #train.loc[ind,'text'] = ','.join([str(TEXT.vocab.stoi[i]) for i in text_windows])
    #test.loc[ind,'label'] = ','.join([str(le.transform([row['event_subtype']]).item())] * len(row['trigger'].split()))
    
train[['text','label']].to_csv('DMCNN/train.csv',index=False)
test[['text','label']].to_csv('DMCNN/test.csv',index=False)
dev[['text','label']].to_csv('DMCNN/dev.csv',index=False)


In [202]:
train,test,dev=data.TabularDataset.splits(
    path="DMCNN",train='train.csv',
    test='test.csv',validation='dev.csv',
    format='csv',fields=[('text', TEXT), ('label', LABEL)],
    skip_header=True
)

F:\Anaconda3\envs\pytorch\lib\site-packages\torchtext\data\example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
F:\Anaconda3\envs\pytorch\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [203]:
TEXT.build_vocab(train,test,dev, vectors=GloVe())

In [204]:
pretrained_embeddings=TEXT.vocab.vectors

In [205]:
BATCH_SIZE = 128
train_iterator, test_iterator, dev_iterator = data.BucketIterator.splits(
    (train, test, dev), 
    batch_size = BATCH_SIZE, 
    sort=False,
    device = DEVICE)

F:\Anaconda3\envs\pytorch\lib\site-packages\torchtext\data\iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [206]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # (batch_size,seq_len,embed,size)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [207]:
class CNN_Block(nn.Module):
    def __init__(self,window_size,out_channel,kernel_size,embedding_dim,drop_out):
        super(CNN_Block,self).__init__()
        self.cnn = nn.Conv2d(in_channels=1,out_channels=out_channel,kernel_size=(kernel_size,embedding_dim),padding=(kernel_size-1,0))
        self.relu = nn.ReLU()
        self.drop_out = nn.Dropout(p=drop_out)
        self.maxpool1d = nn.MaxPool1d(window_size+kernel_size-1)
    
    def forward(self,x):
        cnn_out = self.cnn(x).squeeze(dim=-1)
        relu_out = self.relu(cnn_out)
        drop_out_result = self.drop_out(relu_out)
        return self.maxpool1d(drop_out_result)

In [208]:
class DMCNN(nn.Module):
    def __init__(self,window_sizes,kernel_sizes,out_channel,pretrained_embeddings,drop_out,total_classes):
        super(DMCNN, self).__init__()
        self.window_size = window_size
        self.word_embedding = nn.Embedding.from_pretrained(pretrained_embeddings)
        self.position_encoding = PositionalEncoding(d_model = pretrained_embeddings.size(1),max_len=window_size)
        
        self.cnn_blocks = nn.ModuleList([
            CNN_Block(window_size,
                      out_channel,
                      kernel_size,
                      pretrained_embeddings.size(1),
                      drop_out) for kernel_size in kernel_sizes
        ])
        self.drop_out = nn.Dropout(p=drop_out)
        self.linear = nn.Linear(in_features=out_channel*len(kernel_sizes)+pretrained_embeddings.size(1),out_features=total_classes)
        self.softmax = nn.Softmax()
        
        
    def forward(self, x):
        
        word_embedding = self.word_embedding(x) # (batch_size,win_size,embed_size)
        position_encoding = self.position_encoding(word_embedding) # (batch_size,win_size,embed_size)
        we_pe = position_encoding.unsqueeze(dim=1) # (batch_size,1,win_size,embed_size)
        cnn_outs = []
        for cnn_block in self.cnn_blocks:
            cnn_outs += [cnn_block(we_pe)]
        total_cnn_outs = torch.cat(cnn_outs,dim=2) # (batch_size,out_channel,1)
        total_cnn_outs = total_cnn_outs.view(total_cnn_outs.size(0),-1) #(batch_size,out_channel * kernel number)
        
        #下面是论文中没有提到的trick，把cnn输出和前半段的embedding vector和在一起进linear层
        total_cnn_outs = torch.cat((total_cnn_outs, word_embedding[:, int((window_size-1)/2)]), dim=-1)
        
        #print(total_cnn_outs.shape)
        drop_out_result = self.drop_out(total_cnn_outs)
        y = self.linear(drop_out_result)        
        return y
    

In [209]:
model = DMCNN(31,[2,3,4,5],150,pretrained_embeddings,0.5,34)
model.to(DEVICE)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),)


In [210]:
print(model)

DMCNN(
  (word_embedding): Embedding(6465, 300)
  (position_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (cnn_blocks): ModuleList(
    (0): CNN_Block(
      (cnn): Conv2d(1, 150, kernel_size=(2, 300), stride=(1, 1), padding=(1, 0))
      (relu): ReLU()
      (drop_out): Dropout(p=0.5, inplace=False)
      (maxpool1d): MaxPool1d(kernel_size=32, stride=32, padding=0, dilation=1, ceil_mode=False)
    )
    (1): CNN_Block(
      (cnn): Conv2d(1, 150, kernel_size=(3, 300), stride=(1, 1), padding=(2, 0))
      (relu): ReLU()
      (drop_out): Dropout(p=0.5, inplace=False)
      (maxpool1d): MaxPool1d(kernel_size=33, stride=33, padding=0, dilation=1, ceil_mode=False)
    )
    (2): CNN_Block(
      (cnn): Conv2d(1, 150, kernel_size=(4, 300), stride=(1, 1), padding=(3, 0))
      (relu): ReLU()
      (drop_out): Dropout(p=0.5, inplace=False)
      (maxpool1d): MaxPool1d(kernel_size=34, stride=34, padding=0, dilation=1, ceil_mode=False)
    )
    (3): CNN_Blo

In [211]:
num_epochs = 1000
for epoch in range(1, num_epochs + 1):
    correct = 0
    for batch_idx,batch in enumerate(train_iterator):
        model.train()
        output = model(batch.text.t())
        l = loss(output, batch.label)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        predicted = torch.max(output.data,1)[1]
        correct += (predicted == batch.label).sum()
        
    
    print('epoch %d, loss: %f, correct: %f' % (epoch, l.item(),float(correct)/float((batch_idx+1)*128)))

F:\Anaconda3\envs\pytorch\lib\site-packages\torchtext\data\batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


epoch 1, loss: 1.991436, correct: 0.310268
epoch 2, loss: 1.591162, correct: 0.510491
epoch 3, loss: 1.468250, correct: 0.629018
epoch 4, loss: 1.217354, correct: 0.697545
epoch 5, loss: 0.978770, correct: 0.720313
epoch 6, loss: 0.987584, correct: 0.744420
epoch 7, loss: 0.811315, correct: 0.755357
epoch 8, loss: 0.822719, correct: 0.773214
epoch 9, loss: 0.858468, correct: 0.779687
epoch 10, loss: 0.635328, correct: 0.787277
epoch 11, loss: 0.572548, correct: 0.790625
epoch 12, loss: 0.736246, correct: 0.799777
epoch 13, loss: 0.734063, correct: 0.802455
epoch 14, loss: 0.878417, correct: 0.814286
epoch 15, loss: 0.680614, correct: 0.807813
epoch 16, loss: 0.580156, correct: 0.814063
epoch 17, loss: 0.826607, correct: 0.814286
epoch 18, loss: 0.722678, correct: 0.819866
epoch 19, loss: 0.718393, correct: 0.822321
epoch 20, loss: 0.638322, correct: 0.821205
epoch 21, loss: 0.666372, correct: 0.830580
epoch 22, loss: 0.566107, correct: 0.833929
epoch 23, loss: 0.715182, correct: 0.8294

In [254]:
model.train(False)
correct = 0
l = torch.tensor([],device=DEVICE)
o = torch.tensor([],device=DEVICE)
for batch_idx,batch in enumerate(test_iterator):
    output = model(batch.text.t())
    l = torch.cat([l,batch.label])
    o = torch.cat([o,torch.max(output.data,1)[1]])
    #predicted = torch.max(output.data,1)[1]
    #correct += (predicted == batch.label).sum()

#print('correct: %f' % (float(correct)/float((batch_idx+1)*128)))

F:\Anaconda3\envs\pytorch\lib\site-packages\torchtext\data\batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [255]:
recall_score(o.cpu(),l.cpu(), average='macro')

0.734703522951814

In [256]:
precision_score(o.cpu(),l.cpu(), average='macro')

F:\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6976831593550358

In [257]:
f1_score(o.cpu(),l.cpu(), average='macro')

0.7020610936214675